In [84]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from pypfopt import BlackLittermanModel, plotting
from pypfopt import black_litterman, risk_models, objective_functions
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

In [85]:
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

In [86]:
# Define the list of stock tickers
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']
measurement_days = 10
remaining_days = 30-measurement_days

In [87]:
# Fit non linear ML models to predict stock returns into future

stock_returns = {}
stock_prices = {}
future_returns = {}
short_window_returns = {}

for ticker in tickers:
    data = yf.download(ticker,period="5y")
    print(ticker)
    returns = ((data['Adj Close'] / data['Adj Close'].shift(1))-1).dropna().tolist()
    stock_returns[ticker] = returns

    stock_prices[ticker] = data['Adj Close'].values

    
    prices_x = [returns[i:i+measurement_days] for i in range(len(returns)-remaining_days)]
    prices_y = [returns[i+measurement_days] for i in range(measurement_days,len(returns)-measurement_days)]

    prices_test = [returns[i:i+measurement_days] for i in range(len(returns)-remaining_days,len(returns)-measurement_days)]

    short_window_returns[ticker] = (prices_x, prices_y, prices_test)

    rf = RandomForestRegressor()
    rf.fit(prices_x,prices_y)
    # Predict stock price for 10 future days
    future_returns[ticker] = rf.predict(prices_test)





[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
CCJ
[*********************100%***********************]  1 of 1 completed
COST
[*********************100%***********************]  1 of 1 completed
GOOG
[*********************100%***********************]  1 of 1 completed
GS
[*********************100%***********************]  1 of 1 completed
JPM
[*********************100%***********************]  1 of 1 completed
LLY
[*********************100%***********************]  1 of 1 completed
META
[*********************100%***********************]  1 of 1 completed
MSFT
[*********************100%***********************]  1 of 1 completed
NEE
[*********************100%***********************]  1 of 1 completed
PFE
[*********************100%********************

In [88]:
views = {}
market_caps = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]
    stock = yf.Ticker(ticker)
    market_caps[ticker] = stock.info["marketCap"]
print(market_caps)

{'AAPL': 2914056404992, 'AMD': 173129809920, 'AMZN': 1306456752128, 'CCJ': 12997988352, 'COST': 231952515072, 'GOOG': 1507596042240, 'GS': 104624660480, 'JPM': 406753542144, 'LLY': 429792854016, 'META': 713643393024, 'MSFT': 2443301879808, 'NEE': 150319874048, 'PFE': 216215240704, 'SAP': 155516518400, 'WMT': 417524842496}


In [89]:
stock_prices = pd.DataFrame(stock_prices)
S = risk_models.CovarianceShrinkage(stock_prices).ledoit_wolf()

In [90]:
market_prices = yf.download("SPY", period="max")["Adj Close"]
delta = black_litterman.market_implied_risk_aversion(market_prices)
print(delta)
market_prior = black_litterman.market_implied_prior_returns(market_caps, delta, S)
print(market_prior)

[*********************100%***********************]  1 of 1 completed
2.5716339410761435
AAPL    0.224687
AMD     0.270415
AMZN    0.220906
CCJ     0.139379
COST    0.127587
GOOG    0.207618
GS      0.154056
JPM     0.137532
LLY     0.107819
META    0.243027
MSFT    0.215435
NEE     0.112403
PFE     0.093177
SAP     0.156142
WMT     0.090235
dtype: float64


In [91]:
data = pd.DataFrame()
for ticker in tickers:
    stock = yf.Ticker(ticker)
    df = stock.history(period='5y')  # Retrieve 1-year historical data
    data[ticker] = df['Close']

# Step 2: Calculate historical returns
returns = data.pct_change().dropna()

# Step 3: Calculate statistics
returns_mean = returns.mean()
# Step 4: Assign confidence values based on historical performance
confidence_values = returns_mean.rank() / len(returns_mean)  # Example: Using ranking

# Step 5: Normalize confidence values
confidence_values /= confidence_values.sum()

# Print confidence values
confidence_values = confidence_values.tolist()
print(confidence_values)

[0.10833333333333334, 0.125, 0.041666666666666664, 0.1, 0.08333333333333333, 0.075, 0.03333333333333333, 0.025, 0.11666666666666667, 0.058333333333333334, 0.09166666666666666, 0.06666666666666667, 0.008333333333333333, 0.016666666666666666, 0.05]


In [92]:
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
ret_bl = bl.bl_returns()
print(ret_bl)

# Get cov matrix
S_bl = bl.bl_cov()

rf_ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(0.05, 0.1))
rf_ef.add_objective(objective_functions.L2_reg)
rf_ef.min_volatility()
weights = rf_ef.clean_weights()
print(weights)


AAPL    0.151367
AMD     0.167868
AMZN    0.150293
CCJ     0.086533
COST    0.082817
GOOG    0.139895
GS      0.101957
JPM     0.092037
LLY     0.069686
META    0.164167
MSFT    0.144336
NEE     0.073080
PFE     0.063609
SAP     0.104546
WMT     0.060420
dtype: float64
OrderedDict([('AAPL', 0.05622), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07763), ('GOOG', 0.05952), ('GS', 0.06385), ('JPM', 0.06844), ('LLY', 0.08039), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07866), ('PFE', 0.08389), ('SAP', 0.06698), ('WMT', 0.08802)])


In [103]:
for weight in weights:
    print(weights[weight] * 800000)

44976.0
40000.0
45744.0
49808.0
62104.00000000001
47616.0
51080.0
54752.0
64312.0
40000.0
45568.0
62927.99999999999
67112.0
53584.0
70416.0


In [93]:
future_returns = {}

for ticker in tickers:
    return_movements = np.array([1 if stock_returns[ticker][i+measurement_days] > stock_returns[ticker][i+measurement_days-1] else 0 for i in range(len(stock_returns[ticker])-measurement_days)])
    return_movements = return_movements[measurement_days:]

    logreg = LogisticRegression()
    logreg.fit(short_window_returns[ticker][0], return_movements)

    future_return_movements = logreg.predict(short_window_returns[ticker][2])
    future_returns[ticker] = [stock_returns[ticker][-1] * (1 + movement) for movement in future_return_movements]

In [94]:
views = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]


logreg_bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
logreg_ret_bl = logreg_bl.bl_returns()
print(logreg_ret_bl)

# Get cov matrix
logreg_S_bl = logreg_bl.bl_cov()

logreg_ef = EfficientFrontier(logreg_ret_bl, logreg_S_bl, weight_bounds=(0.05, 0.1))
logreg_ef.add_objective(objective_functions.L2_reg)
logreg_ef.min_volatility()
weights = logreg_ef.clean_weights()
print(weights)

AAPL    0.144534
AMD     0.157840
AMZN    0.142884
CCJ     0.082328
COST    0.079275
GOOG    0.131964
GS      0.097181
JPM     0.088013
LLY     0.065550
META    0.155188
MSFT    0.137067
NEE     0.069908
PFE     0.060698
SAP     0.099495
WMT     0.058263
dtype: float64
OrderedDict([('AAPL', 0.05622), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07763), ('GOOG', 0.05952), ('GS', 0.06385), ('JPM', 0.06844), ('LLY', 0.08039), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07866), ('PFE', 0.08389), ('SAP', 0.06698), ('WMT', 0.08802)])


In [95]:
future_returns = {}

for ticker in tickers:
    return_movements = np.array([1 if stock_returns[ticker][i+measurement_days] > stock_returns[ticker][i+measurement_days-1] else 0 for i in range(len(stock_returns[ticker])-measurement_days)])
    return_movements = return_movements[measurement_days:]

    svm = SVC(kernel='poly')
    svm.fit(short_window_returns[ticker][0], return_movements)

    future_return_movements = svm.predict(short_window_returns[ticker][2])
    future_returns[ticker] = [stock_returns[ticker][-1] * (1 + movement) for movement in future_return_movements]

In [96]:
views = {}
for ticker in tickers:
    views[ticker] = future_returns[ticker][-1]


svm_bl = BlackLittermanModel(S, pi=market_prior, absolute_views=views, omega="idzorek", view_confidences=confidence_values)
# Get expected returns
svm_ret_bl = svm_bl.bl_returns()
print(svm_ret_bl)

# Get cov matrix
svm_S_bl = svm_bl.bl_cov()

svm_ef = EfficientFrontier(svm_ret_bl, svm_S_bl, weight_bounds=(0.05, 0.1))
svm_ef.add_objective(objective_functions.L2_reg)
svm_ef.min_volatility()
weights = svm_ef.clean_weights()
print(weights)

AAPL    0.144315
AMD     0.157743
AMZN    0.143089
CCJ     0.082224
COST    0.079052
GOOG    0.131833
GS      0.096918
JPM     0.087692
LLY     0.064026
META    0.155130
MSFT    0.136812
NEE     0.069533
PFE     0.059914
SAP     0.099259
WMT     0.058012
dtype: float64
OrderedDict([('AAPL', 0.05622), ('AMD', 0.05), ('AMZN', 0.05718), ('CCJ', 0.06226), ('COST', 0.07763), ('GOOG', 0.05952), ('GS', 0.06385), ('JPM', 0.06844), ('LLY', 0.08039), ('META', 0.05), ('MSFT', 0.05696), ('NEE', 0.07866), ('PFE', 0.08389), ('SAP', 0.06698), ('WMT', 0.08802)])


In [98]:
rf_ef.portfolio_performance(verbose=True)
print("\n")
logreg_ef.portfolio_performance(verbose=True)
print("\n")
svm_ef.portfolio_performance(verbose=True)
print("\n")



Expected annual return: 10.4%
Annual volatility: 21.8%
Sharpe Ratio: 0.39


Expected annual return: 9.9%
Annual volatility: 21.8%
Sharpe Ratio: 0.36


Expected annual return: 9.8%
Annual volatility: 21.8%
Sharpe Ratio: 0.36


